In [1]:
import cv2
import numpy as np

In [2]:
img_np = np.ones([5,5])

print(img_np)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [3]:
kernel = np.array([-1, 0, 1 ])
print(kernel)

[-1  0  1]


In [4]:
# 디지터 신호처리의 FIR 필터 한 번 살펴보면 좋음
# cv2는 OpenCV라는 C++로 작성된 라이브러리를 python 가져다 사용
# 그러므로 python의 행렬 정보를 OpenCV용으로 변환할 필요가 있음
# 목적 : 타입 컨버팅

img_cv = cv2.resize(img_np, (5,5))
                    
print(img_cv)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [5]:
kernel = cv2.resize(kernel, (1, 3))
print(kernel)

[[-1]
 [ 0]
 [ 1]]


In [6]:
img_cv[0][0] = 0
img_cv[2][2] = 2
img_cv[4][4] = 3

# filter2D - Convolution이 결국
# 라플라스 변환과 푸리에 변환에 관계를 가지고 있기 때문
# 푸리에 변환이 완료된 두 함수의 곱은 시간 영역에서 컨벌류션과 동일하다.
# 주파수 영역으로 변환한 두 함수의 곱이 시간 영역에서 컨벌루션과 동일하다.
# (증명: 자코비안 행렬과 라플라스 변환이 필요함)
conv_test = cv2.filter2D(img_cv, -1, kernel)

print(conv_test)

[[ 0.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  2.]
 [ 0.  0.  0.  0.  0.]]


In [7]:
import copy

# 포인터를 공유 -> malloc을 통해 해결(C, C++ 관점)
# 객체를 공유 -> new를 통해 해결(자바 관점)
# 파이썬에서 위의 역할을 해주는 것이 copy.deepcopy()
def filter_2d(input_img, kernel):
    input_img = copy.deepcopy(input_img)
    output_img = copy.deepcopy(input_img)
    
    print('input_img.shape[0]:', input_img.shape[0])
    print('input_img.shape[1]:', input_img.shape[1])
    
    for i in range(input_img.shape[0]):
        print(input_img[i])
        
    # 컨벌루션 연산이 적용되는 대상은 무엇인가 ? 답: 이미지
    # 이미지 -> (다차원)행렬 17
    for row in range(input_img.shape[0]):
        output_img[row][0] = input_img[row][1] * kernel[0] + \
                        input_img[row][0] * kernel[1] + \
                        input_img[row][1] * kernel[2]
        
        for col in range(1, input_img.shape[1] - 1):
            output_img[row][col] = input_img[i][col- 1] * kernel[0] + \
                        input_img[row][col] * kernel[1] + \
                        input_img[row][col + 1] * kernel[2]
            
        output_img[row][input_img.shape[1] - 1] = \
            input_img[row][input_img.shape[1] - 2] * kernel[0] + \
            input_img[row][input_img.shape[1] - 1] * kernel[1] + \
            input_img[row][input_img.shape[1] - 2] * kernel[2]
            
    return output_img.T

In [8]:
conv_img = filter_2d(img_cv, kernel)
conv_img2 = cv2.filter2D(img_cv, -1, kernel)

print("")
print(conv_img)
print("")
print(conv_img == conv_img2)

input_img.shape[0]: 5
input_img.shape[1]: 5
[0. 1. 1. 1. 1.]
[1. 1. 1. 1. 1.]
[1. 1. 2. 1. 1.]
[1. 1. 1. 1. 1.]
[1. 1. 1. 1. 3.]

[[0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2.]
 [0. 0. 0. 0. 0.]]

[[ True  True  True  True  True]
 [False  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True False  True  True]
 [ True  True  True  True  True]]


In [9]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [10]:
classifier = VGG16()

In [11]:
classifier.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
suitImg = image.load_img("suit.jpg", target_size=(224, 224))

FileNotFoundError: [Errno 2] No such file or directory: 'suit.jpg'

In [ ]:
transformed_img = image.img_to_array(suitImg)
print(transformed_img.shape)

In [ ]:
transformed_img = np.expand_dims(transformed_image, axis = 0)
print(transformed_img.shape)
print(transformed_img)

In [ ]:
transformed_img = preprocess_input(transformed_img)
print(transformed_img.shape)
print(transformed_img)

In [ ]:
pred = classifier.predict(transformed_img)
print(pred)

In [ ]:
print(pred.shape)

In [ ]:
from keras.applications.vgg16 import decode_predictions

decode_predictions(pred, top = 5)

In [ ]:
label = decode_predictions(pred)

In [ ]:
print('%s (%.2f%%)' % (label[0][0][1], label[0][0][2] * 100))

In [ ]:
print('%s (%.2f%%)' % (label[0][1][1], label[0][1][2] * 100))

In [ ]:
print('%s (%.2f%%)' % (label[0][2][1], label[0][2][2] * 100))

In [ ]:
print('top 5!')

for i in range(5):
    print('%16s (%.2f%%)' % (label[0][i][1], label[0][i][2] * 100))